# EnvTrace Quickstart and Pretty Reports

This notebook demonstrates EnvTrace on:
- A simple discrete example loaded from JSON traces.
- A mixed example with a continuous channel (e.g., temperature).
- Using the CLI to align and score traces.

All outputs use the pretty text report with alignment table and metric breakdown.

## 1) Simple example from JSON traces

We load ground-truth and predicted traces from `envtrace/examples/traces`, ignore the `det:AcquireTime` parameter channel for discrete matching, and print a human-friendly report.

In [3]:
from pathlib import Path
import envtrace
from envtrace.io.json_io import load_trace
from envtrace.core import Evaluator, EvaluateRequest, NumericToleranceComparator
from envtrace.reporting import format_text_report

PKG_DIR = Path(envtrace.__file__).resolve().parent
TRACES_DIR = PKG_DIR / "examples" / "traces"
gt_path = str(TRACES_DIR / "gt.json")
pred_path = str(TRACES_DIR / "pred.json")
gt = load_trace(gt_path)
pred = load_trace(pred_path)

# Ignore AcquireTime for discrete matching (it's a parameter-setting PV)
req = EvaluateRequest(
    gt=gt,
    pred=pred,
    comparator=NumericToleranceComparator(1e-3),
    ignore_channels={"det:AcquireTime"},
    include_structure=True,
)
ev = Evaluator.default()
res = ev.evaluate(req)

print(format_text_report(res, evaluator=ev, req=req, max_rows=30, title="EnvTrace Simple Example"))

EnvTrace Simple Example
Full score: 0.991
Accuracy:   True

Metrics:
- discrete: score=1.000, pass=True
  · value_matches=4/4, mismatch_rate=0.000
- timing: score=0.955, pass=True
  · r2=0.999
  · slope=0.989
  · duration_ratio=1.008
  · mape=0.180
- structure: score=1.000, pass=True
  · gap_rate=0.000
- aggregation contributions:
  · discrete: 0.800
  · timing: 0.191
  · total: 0.991

Decision breakdown:
  require_discrete_exact: True
  timing_metric_name:     timing
  required_metrics:       (none)
  pass[discrete]: True
  pass[timing]: True

Alignment (first rows, discrete events only):
 IDX | GT.channel       |     GT.t | GT.value         | PR.channel       |     PR.t | PR.value         |  MATCH 
----------------------------------------------------------------------------------------------------------------
   1 | motor:x          |    0.000 | 0.0              | motor:x          |    0.000 | 0.0              |    ✓   
   2 | det:Acquire      |    0.100 | 1                | det:Acqu

Example with some PV mismatches

In [11]:
from envtrace.core import (
    Event, Trace, Evaluator, EvaluateRequest,
    ExactEqualityComparator, NumericToleranceComparator,
)

gt = Trace([
    Event("motor:x", 0.00, 0.0),
    Event("det:AcquireTime", 0.05, 1.0),
    Event("det:Acquire", 0.10, 1),
    Event("det:Acquire", 1.10, 0),
    Event("motor:y", 1.2, 30)
])
pred = Trace([
    Event("motor:x", 0.00, 0.0),
    Event("det:AcquireTime", 0.05, 1.0),
    Event("det:Acquire", 0.12, 1),
    Event("det:Acquire", 1.08, 0),
    Event("det:Acquire", 1.10, 1),
    Event("det:Acquire", 1.20, 0),
    Event("motor:y", 1.2, 10)
])

req = EvaluateRequest(
    gt=gt,
    pred=pred,
    comparator=NumericToleranceComparator(1e-3),
    include_structure=True,
)
ev = Evaluator.default()
res = ev.evaluate(req)

print(format_text_report(res, evaluator=ev, req=req, max_rows=30, title="EnvTrace Simple Example"))

EnvTrace Simple Example
Full score: 0.647
Accuracy:   False

Metrics:
- discrete: score=0.571, pass=False
  · value_matches=4/7, mismatch_rate=0.429
- timing: score=0.951, pass=True
  · r2=1.000
  · slope=0.976
  · duration_ratio=0.982
  · mape=0.147
- structure: score=0.714, pass=False
  · gap_rate=0.286
- aggregation contributions:
  · discrete: 0.457
  · timing: 0.190
  · total: 0.647

Decision breakdown:
  require_discrete_exact: True
  timing_metric_name:     timing
  required_metrics:       (none)
  pass[discrete]: False
  pass[timing]: True

Alignment (first rows, discrete events only):
 IDX | GT.channel       |     GT.t | GT.value         | PR.channel       |     PR.t | PR.value         |  MATCH 
----------------------------------------------------------------------------------------------------------------
   1 | motor:x          |    0.000 | 0.0              | motor:x          |    0.000 | 0.0              |    ✓   
   2 | det:AcquireTime  |    0.050 | 1.0              | det:

## 2) Mixed example with a continuous channel

This example constructs traces in-memory including a continuous series (e.g., temperature). We configure a per-channel continuous profile evaluation and pretty-print the full report.

In [4]:
from envtrace.core import (
    Event, Trace, Evaluator, EvaluateRequest,
    ExactEqualityComparator, NumericToleranceComparator,
)
from envtrace.reporting import format_text_report

# Build ground-truth and predicted traces (includes a continuous channel)
gt2 = Trace([
    Event("motor:x", 0.00, 0.0),
    Event("det:AcquireTime", 0.05, 1.0),
    Event("det:Acquire", 0.10, 1),
    Event("det:Acquire", 1.10, 0),
    Event("stage:temp", 0.20, 20.0),
    Event("stage:temp", 0.40, 40.0),
])
pred2 = Trace([
    Event("motor:x", 0.00, 0.0),
    Event("det:AcquireTime", 0.05, 1.0),
    Event("det:Acquire", 0.12, 1),
    Event("det:Acquire", 1.08, 0),
    Event("stage:temp", 0.20, 20.0),
    Event("stage:temp", 0.40, 39.5),
])

comp_map = {
    "det:Acquire": ExactEqualityComparator(),
    "motor:x": NumericToleranceComparator(1e-2),
}
continuous_cfg = {
    "stage:temp": {"mae_scale": 15.0, "final_scale": 15.0, "mae_thresh": 5.0, "final_thresh": 5.0, "weight": 1.0}
}
ignore = {"det:AcquireTime"}

ev2 = Evaluator.default()
req2 = EvaluateRequest(
    gt=gt2,
    pred=pred2,
    comparator=NumericToleranceComparator(1e-3),
    comparators_by_channel=comp_map,
    ignore_channels=ignore,
    continuous_channels=continuous_cfg,
    include_structure=True,
)
res2 = ev2.evaluate(req2)
print(format_text_report(res2, evaluator=ev2, req=req2, max_rows=30, title="EnvTrace Continuous Example"))

EnvTrace Continuous Example
Full score: 0.987
Accuracy:   True

Metrics:
- discrete: score=1.000, pass=True
  · value_matches=3/3, mismatch_rate=0.000
- timing: score=0.956, pass=True
  · r2=1.000
  · slope=0.973
  · duration_ratio=0.982
  · mape=0.120
- continuous: score=0.979, pass=True
  · stage:temp: score=0.979, pass=True
- structure: score=1.000, pass=True
  · gap_rate=0.000
- aggregation contributions:
  · discrete: 0.600
  · timing: 0.191
  · continuous: 0.196
  · total: 0.987

Decision breakdown:
  require_discrete_exact: True
  timing_metric_name:     timing
  required_metrics:       (none)
  pass[discrete]: True
  pass[timing]: True

Alignment (first rows, discrete events only):
 IDX | GT.channel       |     GT.t | GT.value         | PR.channel       |     PR.t | PR.value         |  MATCH 
----------------------------------------------------------------------------------------------------------------
   1 | motor:x          |    0.000 | 0.0              | motor:x          | 

## 3) CLI usage

You can also call the EnvTrace CLI to align and score two JSON traces. Omit `--out` to print a pretty report; include it to write a JSON report to disk.

In [5]:
import subprocess
import json
import sys
from pathlib import Path
import envtrace

PKG_DIR = Path(envtrace.__file__).resolve().parent
TRACES_DIR = PKG_DIR / "examples" / "traces"
REPORTS_DIR = PKG_DIR / "examples" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

gt_path = str(TRACES_DIR / "gt.json")
pred_path = str(TRACES_DIR / "pred.json")
out_json = str(REPORTS_DIR / "cli_report.json")

# Example 1: Pretty report to stdout
print("=" * 80)
print("CLI Example 1: Pretty text report to stdout")
print("=" * 80)
result = subprocess.run(
    [sys.executable, "-m", "envtrace.cli", "align", "--gt", gt_path, "--pred", pred_path],
    capture_output=True, text=True
)
print(result.stdout)

CLI Example 1: Pretty text report to stdout
EnvTrace Report
Full score: 0.991
Accuracy:   True

Metrics:
- discrete: score=1.000, pass=True
  Â· value_matches=5/5, mismatch_rate=0.000
- timing: score=0.954, pass=True
  Â· r2=0.999
  Â· slope=0.991
  Â· duration_ratio=1.008
  Â· mape=0.185
- structure: score=1.000, pass=True
  Â· gap_rate=0.000
- aggregation contributions:
  Â· discrete: 0.800
  Â· timing: 0.191
  Â· total: 0.991

Decision breakdown:
  require_discrete_exact: True
  timing_metric_name:     timing
  required_metrics:       (none)
  pass[discrete]: True
  pass[timing]: True

Alignment (first rows, discrete events only):
 IDX | GT.channel       |     GT.t | GT.value         | PR.channel       |     PR.t | PR.value         |  MATCH 
----------------------------------------------------------------------------------------------------------------
   1 | motor:x          |    0.000 | 0.0              | motor:x          |    0.000 | 0.0              |    âœ“   
   2 | det:Acquir

In [6]:
# Example 2: JSON report to file
print("\n" + "=" * 80)
print("CLI Example 2: JSON report to file")
print("=" * 80)
subprocess.run(
    [sys.executable, "-m", "envtrace.cli", "align", "--gt", gt_path, "--pred", pred_path, "--out", out_json],
    capture_output=True, text=True
)
with open(out_json, "r", encoding="utf-8") as f:
    report = json.load(f)
print(f"Report written to: {out_json}")
print(f"Full score: {report['full_score']:.3f}")
print(f"Accuracy: {report['accuracy']}")


CLI Example 2: JSON report to file
Report written to: C:\Users\noahv\Desktop\BNL\VISION_dev_be\envtrace\examples\reports\cli_report.json
Full score: 0.991
Accuracy: True
